In [1]:
!pip install timm matplotlib

In [2]:
!curl https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/RedCat_8727.jpg/1200px-RedCat_8727.jpg -o cat.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  233k  100  233k    0     0  1171k      0 --:--:-- --:--:-- --:--:-- 1190k


In [6]:
import sys
sys.path.append('../src')
# from open_clip.transform import image_transform
from timm.data import create_transform

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display


def make_grid(images, grid_size=(3, 3), padding=10):
    """
    Create a grid of images.

    :param images: List of PIL Image objects.
    :param grid_size: Tuple indicating grid shape (rows, columns).
    :param padding: Padding between images in grid.
    :return: A single PIL Image object containing the grid.
    """
    img_width, img_height = images[0].size

    # Dimensions for the full grid image
    grid_width = grid_size[1] * img_width + (grid_size[1] - 1) * padding
    grid_height = grid_size[0] * img_height + (grid_size[0] - 1) * padding

    # Create an empty white image to place the thumbnails onto
    grid_image = Image.new('RGB', (grid_width, grid_height), color='white')

    for idx, img in enumerate(images):
        row = idx // grid_size[1]
        col = idx % grid_size[1]
        x = col * (img_width + padding)
        y = row * (img_height + padding)
        grid_image.paste(img, (x, y))
    display(grid_image)

In [7]:
# Load the image using Pillow
image = Image.open('cat.jpg')
# Check if the image has an alpha channel
if image.mode == 'RGBA':
    r, g, b, a = image.split()
    image = Image.merge('RGB', (r, g, b))

In [12]:

def plot_images(scale_lower=0.6, scale_upper=1.0, brightness=0.0, contrast=0.0, saturation=0.0, hue=0.0):
    #t = image_transform(336, True, 0.5, 0.5, aug_cfg = {'use_timm': True, 
    #                                                   'scale': (scale_lower, scale_upper), 
    #                                                   'color_jitter': (brightness, contrast, saturation, hue)})

    mean=0.5
    std=0.5
    image_size = 336
    if not isinstance(mean, (list, tuple)):
        mean = (mean,) * 3
    if not isinstance(std, (list, tuple)):
        std = (std,) * 3
    if isinstance(image_size, (tuple, list)):
        assert len(image_size) >= 2
        input_size = (3,) + image_size[-2:]
    else:
        input_size = (3, image_size, image_size)

    aug_cfg = {'scale': (scale_lower, scale_upper), 
               'color_jitter': (brightness, contrast, saturation, hue),
               'interpolation': 'random',}
    t = create_transform(
                input_size=input_size,
                is_training=True,
                hflip=0.,
                mean=mean,
                std=std,
                re_mode='pixel',
                **aug_cfg,
            )

    
    tensors = []
    images = []
    for _ in range(27):
        tensor = t(image)
        tensor = tensor.permute(1, 2, 0)
        tensor = (tensor + 1)/2
        tensor = (tensor * 255).byte()
        pil_image = Image.fromarray(tensor.numpy())
        tensors += [tensor]
        images += [pil_image]

    make_grid(images, (9, 3))

# Create sliders
brightness_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.25, description='Brightness')
contrast_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.25, description='Contrast')
saturation_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.25, description='Saturation')
hue_slider = widgets.FloatSlider(min=0, max=0.5, step=0.001, value=0.033, description='Hue')
scale_lower_slider = widgets.FloatSlider(min=0, max=2, step=0.01, value=0.66, description='Scale lower')
scale_upper_slider = widgets.FloatSlider(min=0, max=2, step=0.01, value=1.0, description='Scale upper')

# Interactive plot
interactive_plot = interactive(plot_images,
                               scale_lower=scale_lower_slider,
                               scale_upper=scale_upper_slider,
                               brightness=brightness_slider, 
                               contrast=contrast_slider, 
                               saturation=saturation_slider, 
                               hue=hue_slider)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

# 0.66, 1, 0.2, 0.2, 0.2, 0.02 -> all look good
# 0.66, 1, 0.25, 0.25, 0.25, 0.03 -> has some ugly ones

interactive(children=(FloatSlider(value=0.66, description='Scale lower', max=2.0, step=0.01), FloatSlider(valu…